In [2]:
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Converts `SavedModel` to TensorRT graph and measures inference time.

"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
from absl import app
from absl import flags
import requests
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
#import tensorrt as trt
from tensorflow.python.saved_model import loader
from tensorflow.python.saved_model import tag_constants

import IPython.display as Disp
import numpy as np
import cv2

In [4]:

image_url = 'https://tensorflow.org/images/blogs/serving/cat.jpg'
saved_model_dir = 'tensorrt_model_testing/'
model_input = 'input_1:0'
model_output = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',
               'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',
               'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0']


#### loading original tensorflow model

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile('tensorrt_model_testing/saved_model.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

#### converting to tensorrt graph

In [6]:
trt_graph = trt.create_inference_graph(od_graph_def,model_output,precision_mode='FP16')

AttributeError: module 'tensorrt' has no attribute 'create_inference_graph'

In [5]:
trt_detection_graph = tf.Graph()

with trt_detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    tf.import_graph_def(trt_graph,name='')

#### saving tensorrt graph

In [6]:
with trt_detection_graph.as_default():
    with tf.Session(graph=trt_detection_graph) as sess:
        tf.train.write_graph(sess.graph,
                         'tensorrt_model_testing','tensorrt_model_epoch26.pb', as_text=True)

#### normal graph

In [ ]:
cap = cv2.VideoCapture(0)
with trt_detection_graph.as_default():
    
    with tf.Session(graph=trt_detection_graph) as sess:
        while True:
            Disp.clear_output(wait=True)
            for i in range(1):
                ret,image  = cap.read()
            draw = image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

            # process image
            start = time.time()
            op = sess.run(model_output,feed_dict={
                                model_input: [image]})

            #(boxes, scores, labels) = sess.run([boxes, scores,labels],feed_dict={image_tensor: image_np_expanded})
            print("FPS = ",1/(time.time()-start))
cap.release()

In [8]:
cap.release()